# Projet data management : Morbidité hospitalière aux cours des années 2018 - 2022 

Ce projet de Data Management à pour but de filtrer, de traiter et de représenter des données provenant d'une base de la morbidité hospitalière durant les années comprises entre 2018 et 2022. Ce premier tableau représente le taux de recours aux établissements de soins de courte durée (MCO) selon le sexe, l’âge des patients et la pathologie traitée.

## Importation des données

Dans cette première partie, nous allons importer la dase et la représenter sous forme de tableau avec DataFrame. Il est importer de connaître la base de données et donc utiliser des outils de Pandas pour l'intéroger. 

In [ ]:
# importation des outils
import pandas as pd         # permettant la création de fataframe
import json                 # pour lire les jason
import plotly.express as px # pour ploter des figures
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy.stats import norm

In [ ]:
# lecture du csv
df = pd.read_csv('tableau_1.csv', sep = ';')
df.head(20)


In [ ]:
# On controle la dimension du tableau de données
df.shape

In [ ]:
# on affiche les informations du tableau
df.info()

## Filtrage

Dans cette seconde partie nous allons filtrer la base de données, modifier le type object du nbr recours en int afin de pouvoir faire des opérations et traiter les valeurs non disponibles.

In [ ]:
# Cette base de données fait un rassemblement départemental, régional et national. Nous allons les séparer les uns des autres
df_dep = df[df['Niveau']=='Départements']
df_reg = df[df['Niveau']=='Régions']
df_fr = df[df['Niveau']== 'France']

Nous allons étudier le taux de recours aux établissements de soin par département. Donc uniquement utiliser le data frame `df_sejour`

In [ ]:
df_dep.tail(10)

Ici le titre ``ind_freq`` n'est pas très explicite. Cette valeur représente le nombre  de recours aux établissements de santé par département. Nous allons la renomer ``Taux recours``

In [ ]:

df_dep = df_dep.rename(columns={"ind_freq": "nbr recours"})


In [ ]:
# On retire les Dom Tom afin de simplifier la representation cartographique (voir plus loin)
# on retire aussi la somme des recours pour toutes pathologies "TOTAL TOUTES CAUSES"
df_filt_DT_path = df_dep[~df_dep.PATHOLOGIE.str.contains('TOTAL') 
                       & ~df_dep.ZONE.str.contains('971')
                       & ~df_dep.ZONE.str.contains('972')
                       & ~df_dep.ZONE.str.contains('973')
                       & ~df_dep.ZONE.str.contains('974')
                       & ~df_dep.ZONE.str.contains('976')]


In [ ]:
# On souhaiterai faire une différentiation des sexes, on ne conserve donc pas les ensemble "sexe"
# on souhaite garder un dataframe rassemblant les ages mais qui différencie les sexe
df_tot_age = df_filt_DT_path[~df_filt_DT_path['SEXE'].str.contains('Ensemble') & 
                                df_filt_DT_path['Tranche d\'age'].str.contains('Tous âges confondus') ]
df_tot_age = df_tot_age.reset_index(drop=True)


In [ ]:
# Puis un second DataFramme avec le nbr recours par tranche d'âge
df_tranch_age = df_filt_DT_path[df_filt_DT_path['SEXE'].str.contains('Ensemble') & 
                                ~df_filt_DT_path['Tranche d\'age'].str.contains('Tous âges confondus') ]
df_tranch_age = df_tranch_age.reset_index(drop=True)

In [ ]:
df_tot_age.head()

In [ ]:
df_tranch_age.head()

In [ ]:
# Nous faisons un convertion de l'objet nbr recours en int.
df_tot_age['nbr recours'], df_tranch_age['nbr recours']  = pd.to_numeric(
                                df_tot_age['nbr recours'],
                                errors='coerce'   # met NaN si ce n'est pas convertible
                                ), pd.to_numeric(
                                df_tranch_age['nbr recours'],
                                errors='coerce'
                                )

In [ ]:
# vérification des doublons
duplicate_1 = df_tot_age.duplicated()
duplicate_2 = df_tranch_age.duplicated()

In [ ]:
# controle
df_tot_age[duplicate_1]

In [ ]:
# controle
df_tranch_age[duplicate_2]

In [ ]:
# contrôler la présence des NaN après convertion. Présence de valeur ND dans la colonne inf_fred convertits en NaN.
df_tot_age.isna().sum()

In [ ]:
df_tranch_age.isna().sum()

In [ ]:
# Affichage des NaN et en selectionner un pour contrôler
df_tot_age[df_tot_age['nbr recours'].isna()].head()


In [ ]:
# on souhaite appliquer une moyenne. 
# On veut donc voir pour un département ciblé, pour une pathologie précis, 
# si l'application de la moyenne fonctionne correctement.
df_tot_age[
    (df_tot_age["ZONE"] == "71 - Saône-et-Loire") &
    (df_tot_age["PATHOLOGIE"] == "01006-Maladies dues au V.I.H.")
    ]

In [ ]:
# Faisons la moyenne par departement, sexe et pathologie afin de remplacer les NaN par cette moyenne.
# La moyenne n'est pas impéctée par les NaN
df_mean = df_tot_age.groupby(['ZONE', 'SEXE', 'PATHOLOGIE'])['nbr recours'].mean()
df_mean 

In [ ]:
# on applique la moyenne départementale des années antérieurs et postérieurs d'une pathologie pour chaque NaN. On prend en compte le sexe.
df_tot_age['nbr recours'] = df_tot_age.apply(
    lambda row: df_mean.loc[(row['ZONE'], row['SEXE'], row['PATHOLOGIE'])]
                if pd.isna(row['nbr recours']) else row['nbr recours'],
    axis=1
)

In [ ]:
# On contrôle. On voit que la moyenne a bien été appliqué
df_tot_age[
    (df_tot_age["ZONE"] == "71 - Saône-et-Loire") &
    (df_tot_age["PATHOLOGIE"] == "01006-Maladies dues au V.I.H.")
    ]

In [ ]:
# on fait la même avec tout_age
df_tranch_age[df_tranch_age['nbr recours'].isna()].head()


In [ ]:
df_tranch_age[
    (df_tranch_age["ZONE"] == "71 - Saône-et-Loire") &
    (df_tranch_age["PATHOLOGIE"] == "01006-Maladies dues au V.I.H.")
    ].head(20)

In [ ]:
df_mean = df_tranch_age.groupby(['ZONE', 'Tranche d\'age', 'PATHOLOGIE'])['nbr recours'].mean()
df_tranch_age['nbr recours'] = df_tranch_age.apply(
    lambda row: df_mean.loc[(row['ZONE'], row['Tranche d\'age'], row['PATHOLOGIE'])]
                if pd.isna(row['nbr recours']) else row['nbr recours'],
    axis=1
)
df_tranch_age[
    (df_tranch_age["ZONE"] == "71 - Saône-et-Loire") &
    (df_tranch_age["PATHOLOGIE"] == "01006-Maladies dues au V.I.H.")
    ].head()

# On voit aussi que la moyenne à bien été appliquée

Filtrer les valeurs des colonnes comme par exemple extraire uniquement le nom de la pathologie sans son code ou bien séparer le code département avec son nom (important afin de réaliser une représentation carthographique avec plotly.express).

In [ ]:


# Extraire le code département au début de 'ZONE' → '01', '75', '976', etc.
df_tranch_age["dep_code"] = df_tranch_age["ZONE"].str.extract(r"^([0-9A-Z]{2,3})")
df_tot_age["dep_code"]  = df_tot_age["ZONE"].str.extract(r"^([0-9A-Z]{2,3})")
df_tot_age["Département"] = df_tot_age["ZONE"].str.split("-", n=1).str[1].str.strip()
df_tranch_age["Département"] = df_tranch_age["ZONE"].str.split("-", n=1).str[1].str.strip()

# Extraire le nom de la pathologie sans son code
df_tranch_age["Pathologie"] = df_tranch_age["PATHOLOGIE"].str.split("-", n=1).str[1].str.strip()
df_tot_age["Pathologie"] = df_tot_age["PATHOLOGIE"].str.split("-", n=1).str[1].str.strip()

# Extraire la tanche d'âge sans son code
df_tranch_age["Tranche d\'age"] = df_tranch_age["Tranche d\'age"].str.split("-", n=1).str[1].str.strip()



In [ ]:
# On selectionne ce qui nous interesse
df_tot_age = df_tot_age[['ANNEE', 'Département', 'dep_code' ,'Pathologie', 'SEXE', 'nbr recours']]
df_tranch_age = df_tranch_age[['ANNEE', 'Département', 'dep_code', 'Pathologie', 'Tranche d\'age', 'nbr recours']]


Création de 3 variables. variable ``ratio par sexe`` qui represente le pourcentage de recours par rapport 
au nombre total de recours. Variable ``total recours`` qui est la somme des cas par sexe. Variable ``ratio tranche d'age`` qui représente le % du nombre de recours par tranche d'âge.

Pour ce faire il nous faut le total de cas pour une pathologie donnée, une année donnée et 
pour un département donnée.

Cette donnée est présente dans notre tableau initial dp_dept mais elle ne prend pas en compte 
la moyenne que nous venons d'appliquer.

Il nous faut donc calculer ce nouveau total.

In [ ]:


# On choisit les colonnes qui définissent un groupe
group_cols = ["Département", "ANNEE", "Pathologie"]

# somme des sexes par (ZONE, ANNEE, PATHOLOGIE)
df_tot_age["total cas"] = df_tot_age.groupby(group_cols)["nbr recours"].transform("sum")

# pourcentage par sexe dans ce total
df_tot_age["ratio par sexe"] = (df_tot_age["nbr recours"] / df_tot_age["total cas"]) * 100

# somme des sexes par (ZONE, ANNEE, PATHOLOGIE)
df_tranch_age["total cas"] = df_tranch_age.groupby(group_cols)["nbr recours"].transform("sum")

#idem pour les tranches d'âge
df_tranch_age["ratio par tranche d\'age"] = (df_tranch_age["nbr recours"] / df_tranch_age["total cas"]) * 100



In [ ]:
df_tranch_age.head()

In [ ]:
df_tot_age.head(11)

In [ ]:
# Ecriture des tableaux en format csv afin de les utiliser dans l'application streamlit
df_tot_age.to_csv('df_tot_age.csv')
df_tranch_age.to_csv('df_tranch_age.csv')

## Représentation graphique

Dans cette partie, nous allons faire de la représentation graphique à partir de nos 2 dataframes `df_tot_age` et `df_tranch_age`

In [ ]:
# Nous souhaitons faire une représentation graphique du nombre de cas par département. 
# Nous avons donc récupéré un geojson afin d'exploiter cette idée
with open("departements.geojson", encoding="utf-8") as f:
    dep_geojson = json.load(f)

In [ ]:


# Filtrer la pathologie choisie
conditions = {
    "Pathologie": "Maladies infectieuses et parasitaires",
    "Département" : "Ain",
    "ANNEE" : 2019
}

df_p = df_tot_age[(df_tot_age["Pathologie"] == conditions["Pathologie"])].copy()


# Figure choropleth animée
fig = px.choropleth(
    df_p,
    geojson = dep_geojson,
    locations="dep_code",              
    featureidkey="properties.code",    
    color="nbr recours",
    animation_frame="ANNEE",           
    color_continuous_scale="Blues",
    range_color=(0, df_p["nbr recours"].max()),
    labels={"nbr recours": "nbr recours"},
    hover_name="Département",
    hover_data={"Pathologie": True, "Département": False},
    width=600,         
    height=600
)

fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.update_layout(
    title=f"Nombre de cas de {conditions['Pathologie']} par département (H+F)",
    margin={"r":0,"t":40,"l":0,"b":0})

fig.show()

In [ ]:
# on copie la liste
df_tot_age_filt = df_tot_age.copy()
 
# on selectionne ce qui nous interesse
for col, val in conditions.items():
    df_tot_age_filt = df_tot_age_filt[df_tot_age_filt[col] == val]

# idem avec les tranches d'âge
df_tranche_filt = df_tranch_age.copy()
for col, val in conditions.items():
    df_tranche_filt = df_tranche_filt[df_tranche_filt[col] == val]



In [ ]:
# Histogramme du taux de recours selon une pathologie, un département pour l'année 2019
fig = px.bar(
    df_tot_age_filt,
    y='nbr recours', 
    x= 'SEXE',
    color='SEXE', 
    title=f'nbr recours aux établissement de santé pour la pathologie<br>{conditions["Pathologie"]}<br>dans le département {conditions["Département"]} ', 
    labels={
        'nbr recours': 'nombre de cas', 
        'ANNEE': 'Année'}, 
    barmode='group', 
    text='ratio par sexe',
    width=600,         
    height=400,        
    color_discrete_map={ 
        "Homme" : "#318CE7", 
        "Femme" : "#DE3163" 
    }
)
y_range = df_tot_age_filt['nbr recours'].max()*(1.1)
fig.update_traces(
    texttemplate='%{text:.2f}%', 
    textposition='outside')
fig.update_traces(width=0.3)  
fig.update_layout(
    bargap=1,       
)
fig.update_layout(legend_title_text="Légende")
fig.update_yaxes(range=[0, y_range])
fig.show()

In [ ]:
# Histogramme du nombre de séjour pour une pathologie donnée, pour un département donnée selon les années

fig = px.bar(
    df_tranche_filt,
    y='nbr recours', 
    x= 'Tranche d\'age',
    color='Tranche d\'age', 
    title=f'Nombre de séjours de {conditions["Pathologie"]} par tranche d\'âge dans le département {conditions["Département"]} ', 
    labels={
        'nbr recours': 'nombre de cas', 
        'Tranche d\'âge': 'Tranche d\'âge'}, 
    barmode='group', 
    text='ratio par tranche d\'age',      
)
y_range = df_tranche_filt['nbr recours'].max()*(1.1)

fig.update_layout(legend_title_text="Légende")
fig.update_traces(
    texttemplate='%{text:.2f}%', 
    textposition='outside')
fig.update_traces(width=0.6)  
fig.update_yaxes(range=[0, y_range])
fig.show()

## Durée des séjours

Nous voulons maintenant nous intérésser à la durée des séjours selon la pathologie. Nous allons donc nous intérésser au tableau 2. 

In [ ]:
# lecture du csv tableau_2
df_sejour = pd.read_csv('tableau_2.csv', sep = ';')
df_sejour.head()

Nous voyons déjà qu'il y a une concordance entre l'ensemble des hospitalisation de ce tableau et l'ensemble des hospitalisations du tableau 1 que nous avons précédemment traité. Par exemple pour l'année 2018, dans le département de l'Ain, pour la pathologie Maladies infectieuses et parasitaires, nous avons 2164 ce qui est identite au tableau df_tot_age (première ligne).

Comme dans le tableau 1, nous allons nous intéresser uniquement au département et mettre de coté les Dom Tom. Pour les consultation inférieur à 24, nous allons uniquement conserver l'ensemble dfe celle ci (donc ejecter les 2 différentiations "<24h hospitalisations programmées" et "<24h autres hospitalisations").

In [ ]:
df_sejour = df_sejour[df_sejour['Niveau']=='Départements']
df_sejour = df_sejour[~df_sejour.ZONE.str.contains('971')
                       & ~df_sejour.ZONE.str.contains('972')
                       & ~df_sejour.ZONE.str.contains('973')
                       & ~df_sejour.ZONE.str.contains('974')
                       & ~df_sejour.ZONE.str.contains('976')]
df_sejour = df_sejour.drop(columns=[
    "<24h hospitalisations programmées",
    "<24h autres hospitalisations",
    "Niveau",
    "Données"
])


In [ ]:
# On change le nom de la colonne "24h ensemble ..." en "24h "
df_sejour = df_sejour.rename(columns={"<24h ensemble des hospitalisations": "<24h"})

In [ ]:
df_sejour.head()

In [ ]:
# on crée une liste des colonne à convertir (oblect -> float)
object_cols = [
    '<24h',
'1 jour',
'2 jours',
'3 jours',
'4 jours',
'5 jours',
'6 jours',
'7 jours',
'8 jours',
'9 jours',
'10 à 19 jours',
'20 à 29 jours',
'30 jours et plus',
'Ensemble des hospitalisations',
' Durée moyenne de séjour (en jours)']

In [ ]:
# application de la conversion object -> numérique
df_sejour[object_cols] = df_sejour[object_cols].apply(pd.to_numeric, errors="coerce")



In [ ]:
# Extraire le code département au début de 'ZONE' → '01', '75', '976', etc.
df_sejour["dep_code"] = df_sejour["ZONE"].str.extract(r"^([0-9A-Z]{2,3})")
df_sejour["Département"] = df_sejour["ZONE"].str.split("-", n=1).str[1].str.strip()

# Extraire le nom de la pathologie sans son code
df_sejour["Pathologie"] = df_sejour["PATHOLOGIE"].str.split("-", n=1).str[1].str.strip()


In [ ]:
df_sejour.head()

On voudrait faire quelques comparaisons et statistiques sur les durées des séjours selon l'année, le département et selon une pathologie

In [ ]:
for c in df_sejour.columns:
    print(repr(c))

In [ ]:
# Examinons la distribution de la durée du séjour selon la pathologie Maladie infectieuses et parasitaires, 
# dans le département de l'Ain en 2019
conditions = {
    "Pathologie": "Maladies infectieuses et parasitaires",
    "Département" : "Ain",
    "ANNEE" : 2019
}

Nous souhaitons crée des lignes pous chaques tranches de durée de séjour afin de simplifier les calculs et la représentation graphique.

In [ ]:
# Nous récupérons les titres des colonnes qui nous interesse.
duree_cols = [
    '<24h',
'1 jour',
'2 jours',
'3 jours',
'4 jours',
'5 jours',
'6 jours',
'7 jours',
'8 jours',
'9 jours',
'10 à 19 jours',
'20 à 29 jours',
'30 jours et plus']

In [ ]:
# Nous injectons les lignes durée de séjour dans la cononne "Durée du séjour" et mettons le nombre de séjours à coté
df_sejour = df_sejour.melt(
    id_vars=["ANNEE", "Pathologie", "dep_code", "Département", "Ensemble des hospitalisations", " Durée moyenne de séjour (en jours)"],   # ce qu’on garde
    value_vars=duree_cols,                     # LEs durées
    var_name="Durée séjour",                   # nom colonne Durée séjour
    value_name="Nombre séjours"                # Nombre de séjours
)

In [ ]:
# Nous souhaitons un ratio du nombre de séjours par rapport au nombre total d'hospitalisations
df_sejour["ratio durée du séjour"] = (df_sejour["Nombre séjours"] / df_sejour["Ensemble des hospitalisations"]) * 100


Nous souhaitons aussi faire une représentation de la distribution du nombre de séjour sous forme de courbe de gaus. Pour ce faire nous devons attribuer une valeur numérique à chaque catégorie. Pour les classes larges (10–19 jours etc.), on prend le milieu de l’intervalle.

In [ ]:
# Nous allons créer une variable numérique pour la durée du séjour dans la colonne "mapping_duree"
mapping_duree = {
    "<24h": 0.5,
    "1 jour": 1,
    "2 jours": 2,
    "3 jours": 3,
    "4 jours": 4,
    "5 jours": 5,
    "6 jours": 6,
    "7 jours": 7,
    "8 jours": 8,
    "9 jours": 9,
    "10 à 19 jours": 14.5,   # milieu
    "20 à 29 jours": 24.5,
    "30 jours et plus": 35   # valeur arbitraire mais utile
}

df_sejour["Durée_num"] = df_sejour["Durée séjour"].map(mapping_duree)

In [ ]:
# Ecriture du tableau en format csv afin de l'utiliser dans l'application streamlit
df_sejour.to_csv('df_sejour.csv')

In [ ]:
# Afin de simplifier la sortie, nous faisons un filtre pour une pathologie donnée, 
# une année donnée et un département donné.
df_sejour_filt = df_sejour[
    (df_sejour["Département"] == conditions["Département"]) &
    (df_sejour["Pathologie"] == conditions["Pathologie"]) &
    (df_sejour["ANNEE"] == conditions["ANNEE"])
    ]
df_sejour_filt.head()

#### Représentations graphiques

Intéressons nous aux représentations graphiques de cette distribution du nombre de séjour selon les différentes tranches de durée.

In [ ]:
fig = px.bar(
    df_sejour_filt,
    y='Nombre séjours', 
    x='Durée séjour',
    color='Durée séjour', 
    title=f"Nombre de séjours pour la pathologie<br>{conditions['Pathologie']}<br>selon la tranche de durée dans le département {conditions['Département']}",
    labels={
        'Nombre séjours': 'Nombre de séjours', 
        'Durée séjour': 'Durée du séjour'
    },
    barmode='group',
    text='ratio durée du séjour',      
)

fig.update_yaxes(range=[0, df_sejour_filt['Nombre séjours'].max() * 1.1]) 
# facteur 1.1 afin pour améliorer la lisibilité
fig.update_traces(
    texttemplate='%{text:.2f}%', 
    textposition='outside')
fig.update_layout(legend_title_text="Durée du séjour")
fig.update_traces(width=0.6)

fig.show()


Interessons nous à la distribution sous forme de courbe gausienne. Calculons alors la moyenne et l'écart type. Ici notre moyenne prend en compte les séjours inférieur à 24h (ce qui n'a pas été le cas dans la moyenne fournit par la base de donnée).

In [ ]:
x = df_sejour_filt["Durée_num"]
w = df_sejour_filt["Nombre séjours"]

mu = np.average(x, weights=w)
sigma = np.sqrt(np.average((x - mu)**2, weights=w))


x_curve = np.linspace(min(x), max(x), 300)
y_curve = norm.pdf(x_curve, mu, sigma)
y_curve = y_curve * w.sum() / y_curve.sum()   


fig = go.Figure()
fig.add_trace(go.Scatter(x=x_curve, y=y_curve, mode="lines",
                         name="Courbe normale", line=dict(color="black", width=3)))

fig.add_vline(x=mu, line_dash="dash", line_color="red", name="Moyenne")
fig.add_vline(x=mu + sigma, line_dash="dot", line_color="blue")

fig.add_annotation(x=mu, y=max(y_curve)*0.95,
                   text=f"µ = {mu:.2f} j", showarrow=False, font=dict(color="red", size=14))

fig.add_annotation(x=mu + sigma, y=max(y_curve)*0.85,
                   text=f"µ + σ = {mu + sigma:.2f} j", showarrow=False, font=dict(color="blue"))

fig.update_layout(
    title=f"Distribution normale théorique<br>{conditions['Pathologie']} – {conditions['Département']}",
    xaxis_title="Durée du séjour (jours)",
    yaxis_title="Pourcentage du nombre d'hospitalisations (%)",
    template="plotly_white"
)

fig.show()


## WordCloud

Nous allons maintenant nous concentrer sur le text mining d’un article de presse lié à nos données, afin d’identifier les mots-clés principaux. Le texte sera filtré, lemmatisé, puis les fréquences des termes seront visualisées sous forme de WordCloud.

Nous allons charger l'ensemble des librairies Python et le modèle linguistique français de `spaCy` pour préparer les opérations d'analyse sémantique.

In [ ]:
# Installation des librairies
# !pip install unidecode wordcloud nltk spacy
# !python -m spacy download fr_core_news_md

import re
from unidecode import unidecode
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import defaultdict
import spacy

# Chargement du modèle français de spaCy. Modèle md qui est plus précis pour la lemmatisation et la reconnaissance des mots
nlp = spacy.load("fr_core_news_md")

In [ ]:
# Article de presse provenant de https://www.lesechos.fr/economie-france/social/cinq-ans-apres-le-covid-lactivite-hospitaliere-retrouve-des-couleurs-2154382
article = """
Cinq ans après le Covid, l'activité hospitalière « retrouve des couleurs »
Le nombre de séjours à l'hôpital a augmenté de 3,7 % en 2024, selon le baromètre annuel de la Fédération hospitalière de France, permettant de résorber une partie du retard pris pendant la crise sanitaire.
L'activité hospitalière « retrouve des couleurs ». Dans la deuxième édition de son baromètre sur l'accès aux soins publiée ce lundi, cinq ans jour pour jour après l'entrée en vigueur du premier confinement pour faire face à l'épidémie de Covid, la Fédération hospitalière de France (FHF) estime qu'il y a eu 516.000 séjours à l'hôpital de plus qu'attendu en 2024.
Le nombre de séjours hospitaliers a augmenté de 3,7 %, et même de 4,6 % à l'hôpital public. Pour la première fois depuis 2020, « la dette de santé publique commence à se résorber », s'est félicité son président, Arnaud Robinet.
Entre 2019 et 2023, compte tenu notamment des perturbations liées au Covid, quelque 3,5 millions de séjours hospitaliers n'avaient pas pu être réalisés. Une situation qui serait synonyme de retard dans la prise en charge de certains cancers et du suivi des personnes âgées. La reprise constatée l'an dernier concerne toutes les classes d'âges à l'exception des plus de 85 ans qui continuent d'être en situation de sous recours (-6 % par rapport aux niveaux attendus).

Un retard important dans les chirurgies lourdes
Certaines disciplines, comme la neurologie, la rhumatologie, le cardio-vasculaire ou les prises en charge digestives restent cependant en difficulté. « Elles représentent un tiers des activités de médecine, pour un total de 180.000 séjours non réalisés », a indiqué la FHF. Un sous-recours est également constaté pour les chirurgies lourdes pour toutes les classes d'âges confondues. « Ce sont en tout 700.000 séjours de chirurgie qui n'ont pas été réalisés depuis 2020 », a-t-elle également alerté.
Autre point positif, dans son baromètre, la FHF constate que 37 % d'hôpitaux se sont déclarés « en tension » en 2024, soit une quarantaine de moins que l'année précédente. Ils sont également une cinquantaine d'hôpitaux en moins à avoir déclenché des plans blancs, ce protocole prévu par les autorités de santé pour faire face à ces situations exceptionnelles. Le rythme des fermetures de lits diminue aussi et de nombreux établissements anticipent des réouvertures dans certains secteurs en 2025.

« Tous les voyants sont au rouge écarlate »
Pour autant, « cette amorce de rémission n'efface pas la dégradation continue de l'accès aux soins des dernières années », a prévenu Arnaud Robinet. Le président de la FHF a notamment rappelé que « sur le plan financier, à l'hôpital comme dans les Ehpad publics, tous les voyants sont au rouge écarlate », avec un déficit atteignant 2,8 milliards d'euros fin 2024. Et l'accès aux soins se dégrade : selon un sondage Ipsos commandé par la FHF, plus de deux tiers des répondants déclarent avoir renoncé à au moins un acte de soins ces cinq dernières années. Deux sur trois (65 %) disent aussi « avoir peur d'être hospitalisés » au vu de la situation actuelle.
« Nous sommes à un tournant […], ou bien nous changeons de logiciel pour se donner les moyens d'amplifier la reprise », ou nous risquons « de voir notre système de santé s'affaiblir encore davantage », a-t-il averti.
En matière de financement, celui qui est aussi le maire (Horizons) de Reims a appelé l'Etat à soutenir cette « reprise ». Il propose la création d'un « Livret H », sur le modèle du Livret A utilisé pour le logement social. Ou encore un fonds vert destiné uniquement aux hôpitaux. Il remettra « aux pouvoirs publics en mai un cadre de loi de programmation en santé » car « le besoin d'une planification en santé n'a jamais été aussi urgent. »
"""

print(f"Longueur du texte : {len(article)} caractères.")

Nous allons créer une fonction pour identifier et marquer les noms de personnes afin de les exclure, car ils ne font pas partie des mots-clés thématiques. Nous veillerons également à nettoyer le texte pour le rendre homogène, en supprimant majuscules, chiffres et ponctuation, tout en conservant les accents nécessaires à la lemmatisation.

In [ ]:
# Fonction d'anonymisation basée sur spaCy
# La variable 'nlp' est chargée dans le Bloc 1
def anonymiser(texte):
    # Traitement du texte par le modèle spaCy (MD)
    doc = nlp(texte)
    resultat = texte
    
    # Remplacement de toutes les entités de type 'PER' (Personne)
    for ent in doc.ents:
        if ent.label_ == "PER":
            # Utilise la méthode de remplacement classique sur le texte original
            resultat = resultat.replace(ent.text, "nom_prenom")
            
    return resultat

In [ ]:
# Application de l'anonymisation sur l'article
text_names_replaced = anonymiser(article) 

# On retire la chaîne de caractères "nom_prenom" car elle nous intéresse pas
text_names_replaced = text_names_replaced.replace('nom_prenom', '')

# Mise en minuscule
text_lower = text_names_replaced.lower()

# Variable contenant les accents
text_with_accents = text_lower

print(text_with_accents[:1000] + "...") # On affiche les 1000 premiers caractères

In [ ]:
# Suppression des nombres (car ils sont très présents et, si nous les avions remplacés par 'annee' (ou 'nombre'), ces mots seraient devenu les plus 
# fréquents du texte, apparaissant en très grand dans le WordCloud)
text_no_numbers = re.sub(r'[0-9]+', '', text_with_accents)

# Suppression de la ponctuation et caractères spéciaux (ne garder que les lettres a-z) (SAUF les accents)
# Le motif mis à jour autorise toutes les lettres minuscules (a-z) ET les accents français communs.
text_clean_chars = re.sub(r'[^a-zàâéèêëïîôöùûüÿçœæ\s]', ' ', text_no_numbers)

# Suppression des espaces multiples créés par les remplacements précédents
text_clean = re.sub(r'\s+', ' ', text_clean_chars).strip()

print(text_clean[:200] + "...") # On affiche les 200 premiers caractères

Nous voulons conserver que les noms communs et adjectifs significatifs, en excluant les noms propres, verbes, adverbes et stopwords.

In [ ]:
# Tokenisation et Filtrage par Catégorie Grammaticale (POS)

# Traitement du texte nettoyé avec spaCy (text_clean contient les accents)
doc = nlp(text_clean)

# Liste des POS tags (catégories grammaticales) à CONSERVER
# On cible uniquement les Noms Communs et les Adjectifs
pos_to_keep = {'NOUN', 'ADJ'} # Noms Propres (PROPN) sont retirés ici

# Liste des stopwords de spaCy
final_stop_words = set(nlp.Defaults.stop_words) 

# Filtrage : le token est un mot-clé (NOUN/ADJ) ET n'est pas un stopword ET a plus de 3 lettres
tokens_filtered_for_lemmatization = []
for token in doc:
    token_text = token.text
    
    # Le filtre POS est appliqué
    if (token_text not in final_stop_words and 
        len(token_text) > 3 and 
        token.pos_ in pos_to_keep):
        
        tokens_filtered_for_lemmatization.append(token_text)

print(f"Les Noms Propres, Verbes et Adverbes ont été retirés.")
print(f"Nombre de tokens (Noms Communs/Adjectifs) conservés : {len(tokens_filtered_for_lemmatization)}")

Nous allons lemmatiser le texte afin de réduire les mots à leur forme de base, ce qui permettra d’uniformiser le vocabulaire et d’éviter les doublons liés aux variations grammaticales. Ensuite, nous calculerons la fréquence de chaque lemme pour identifier les mots les plus récurrents dans le texte. Ces informations serviront à générer un WordCloud clair et représentatif des termes les plus importants.

In [ ]:
# Lemmatisation

# Permet à spaCy de traiter la séquence (qui contient les accents)
text_for_lemmatization = " ".join(tokens_filtered_for_lemmatization)

# Traitement du texte avec spaCy
doc_lemmatized = nlp(text_for_lemmatization)

# Extraction du lemme pour chaque token
tokens_lemmatized_raw = [token.lemma_ for token in doc_lemmatized]

# Normalisation (retrait des accents) des lemmes pour le WordCloud
tokens_lemmatized = [unidecode(word) for word in tokens_lemmatized_raw]

print(f"Aperçu des 10 premiers lemmes : {tokens_lemmatized[:10]}")

In [ ]:
# Dictionnaire de fréquences
freq = defaultdict(int)
for word in tokens_lemmatized:
    freq[word] += 1

# Tri par fréquence décroissante pour afficher le Top 10
sorted_freq = dict(sorted(freq.items(), key=lambda x: x[1], reverse=True))

print("Top 10 des mots identifiées par Fréquence :")
count = 0
for k, v in sorted_freq.items():
    print(f"Mots : '{k}' \t| Fréquence : {v}") 
    count += 1
    if count == 10: break

In [ ]:
# Création du WordCloud
wordcloud = WordCloud(
    width=1000, 
    height=600, 
    background_color='white', 
    colormap='viridis',          
    max_words=13,    # Limité à 13 mots          
    min_font_size=10,         
    random_state=42           
).generate_from_frequencies(freq)

# Affichage avec Matplotlib
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud", fontsize=16, pad=20) 
plt.show()